#### 1.
K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit) Download the dataset regarding USA House Price Prediction from the 
following link: https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing
Load the dataset and Implement 5- fold cross validation for multiple linear regression (using least square error fit).

Steps:

a) Divide the dataset into input features (all columns except price) and output variable (price)

b) Scale the values of input features.

c) Divide input and output features into five folds.

d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted 
values, and R2_score for each iteration using least square error fit.

e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

def compute_beta(X, y):
    X = np.c_[np.ones(X.shape[0]), X]
    return np.linalg.inv(X.T @ X) @ (X.T @ y)

data = pd.read_csv('datasets/USA_Housing.csv')

y = data['Price'].values
X = data.drop('Price', axis=1).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_vals, beta_list = [], []
for tr_idx, ts_idx in kf.split(X_scaled):
    X_tr, X_ts = X_scaled[tr_idx], X_scaled[ts_idx]
    y_tr, y_ts = y[tr_idx], y[ts_idx]
    
    beta = compute_beta(X_tr, y_tr)
    beta_list.append(beta)
    
    X_ts_b = np.c_[np.ones(X_ts.shape[0]), X_ts]
    preds = X_ts_b @ beta
    score = r2_score(y_ts, preds)
    r2_vals.append(score)
    
    print("R2:", round(score, 4))

best_idx = np.argmax(r2_vals)
best_beta = beta_list[best_idx]
print("\nBest Beta Coefficients:", best_beta)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

X_test_b = np.c_[np.ones(X_test.shape[0]), X_test]
y_pred = X_test_b @ best_beta
r2_final = r2_score(y_test, y_pred)

print(f"\nFinal R2 Score (on 30% test data): {r2_final:.4f}")

R2: 0.918
R2: 0.9146
R2: 0.9116
R2: 0.9193
R2: 0.9244

Best Beta Coefficients: [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]

Final R2 Score (on 30% test data): 0.9147


#### 2.
Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization) Consider the same dataset of Q1, rather than dividing the
dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%). Consider four different values of 
learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations. For each 
set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients.

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

def grad_descent(X, y, alpha, steps):
    m, n = X.shape
    X_ = np.c_[np.ones(m), X]
    beta = np.zeros(n + 1)
    for i in range(steps):
        gradient = (2/m) * X_.T.dot(X_.dot(beta) - y)
        beta -= alpha * gradient
    return beta

data = pd.read_csv('datasets/USA_Housing.csv')

y = data['Price'].values
X = data.drop('Price', axis=1).values

sc = StandardScaler()
X_scaled = sc.fit_transform(X)

# 56% train, 14% val, 30% test
X_tr, X_temp, y_tr, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_te, y_val, y_te = train_test_split(X_temp, y_temp, test_size=0.68, random_state=42)

alphas = [0.0001, 0.001, 0.01]
steps = 2000
outcomes = []

for a in alphas:
    print(f"\nRunning Gradient Descent with α = {a}")
    beta = grad_descent(X_tr, y_tr, a, 1000)

    Xv_b = np.c_[np.ones(X_val.shape[0]), X_val]
    preds_val = Xv_b @ beta
    r2_val = r2_score(y_val, preds_val)

    Xt_b = np.c_[np.ones(X_te.shape[0]), X_te]
    preds_test = Xt_b @ beta
    r2_test = r2_score(y_te, preds_test)

    outcomes.append({'alpha': a, 'r2_val': r2_val, 'r2_test': r2_test, 'beta': beta})

    print(f"Validation R²: {round(r2_val, 4)}")
    print(f"Test R²: {round(r2_test, 4)}")

best = max(outcomes, key=lambda d: d['r2_val'])
print("\nBest Model (based on Validation R²):")
print(f"Learning Rate: {best['alpha']}")
print("Coefficients (β):", best['beta'])
print(f"R² on Validation: {best['r2_val']:.4f}")
print(f"R² on Test: {best['r2_test']:.4f}")



Running Gradient Descent with α = 0.0001
Validation R²: -8.579
Test R²: -8.2496

Running Gradient Descent with α = 0.001
Validation R²: 0.6467
Test R²: 0.6531

Running Gradient Descent with α = 0.01
Validation R²: 0.9202
Test R²: 0.9133

Best Model (based on Validation R²):
Learning Rate: 0.01
Coefficients (β): [1232180.23348518  230645.80966846  165328.83294952  120043.26539079
    2946.81725564  151375.2805497 ]
R² on Validation: 0.9202
R² on Test: 0.9133


#### Pre-processing and Multiple Linear Regression
Download the dataset regarding Car Price Prediction from the following link:
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

1. Load the dataset with following column names ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"] and replace all ? values with NaN

2. Replace all NaN values with central tendency imputation. Drop the rows with NaN values in price column

3. There are 10 columns in the dataset with non-numeric values. Convert these values to numeric values using following scheme:
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures for e.g., two to 2
(ii) For "body_style", "drive_wheels": use dummy encoding scheme
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding scheme
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.

4. Divide the dataset into input features (all columns except price) and output variable (price). Scale all input features.

5. Train a linear regressor on 70% of data (using inbuilt linear regression function of Python) and test its performance on remaining 30% of data.

6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of Python). Does it lead to any performance improvement on test set?

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

cols = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", "body_style", "drive_wheels", "engine_location", 
        "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", 
        "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    names=cols, na_values='?'
)

for c in ['normalized_losses', 'horsepower', 'peak_rpm', 'price', 'bore', 'stroke']:
    imp = SimpleImputer(strategy='mean')
    df[c] = imp.fit_transform(df[[c]])

df = df.dropna(subset=['price'])

door_map = {'two': 2, 'four': 4}
df['num_doors'] = df['num_doors'].map(door_map).fillna(4)

cyl_map = {'two': 2, 'three': 3, 'four': 4, 'five': 5,
           'six': 6, 'eight': 8, 'twelve': 12}
df['num_cylinders'] = df['num_cylinders'].map(cyl_map)

df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)

lbl = LabelEncoder()
for c in ['make', 'aspiration', 'engine_location', 'fuel_type']:
    df[c] = lbl.fit_transform(df[c])

df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if 'pfi' in str(x) else 0)
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if 'ohc' in str(x) else 0)

X = df.drop('price', axis=1)
y = df['price']

sc = StandardScaler()
X_s = sc.fit_transform(X)

X_tr, X_te, y_tr, y_te = train_test_split(X_s, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_tr, y_tr)
pred = model.predict(X_te)
r2_base = r2_score(y_te, pred)
print(f"R² without PCA: {r2_base:.4f}")

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_s)

Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(X_pca, y, test_size=0.3, random_state=42)

model_pca = LinearRegression()
model_pca.fit(Xp_tr, yp_tr)
pred_pca = model_pca.predict(Xp_te)
r2_pca = r2_score(yp_te, pred_pca)
print(f"R² after PCA: {r2_pca:.4f}")

if r2_pca > r2_base:
    print("\nPCA improved model performance.")
elif r2_pca < r2_base:
    print("\nPCA slightly reduced model performance.")
else:
    print("\nPCA had no effect on performance.")


R² without PCA: 0.8044
R² after PCA: 0.7501

PCA slightly reduced model performance.
